In [89]:
import io
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import sys

In [117]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.epsgtools as epsg
import posarmctools.ekfnavtools as ekf

In [91]:
from posarmctools.readdata import check_ini, getInfoRecords, checkTimestamps

In [92]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Check ini file

In [ ]:
filename = "/home/pleroy/DATA/PoSAR-X/PIMA-1/flight1.ini"
dir_sbg, session, sbg_hours, dir_posar, day, hours = check_ini(filename)

# Check recorded file

In [ ]:
selectedRecord = 0

In [ ]:
data_dir = os.path.join(dir_posar, day, f"{day}_{hours[selectedRecord]}")
num, idx = getInfoRecords(data_dir, "record", "bin")

# Check timestamps

In [ ]:
day_hour = day + "_" + hours[0]
timestamps_filename = os.path.join(dir_posar, day, day_hour, f"{day_hour}_timeStamps.data")
bufferNumber, timestamp = checkTimestamps(timestamps_filename, day_hour)

In [ ]:
tuple('ABCD')

In [ ]:
list(range(10))

In [ ]:
iterable = 'ABCD'
r = 2
def myIter(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    print("!!!")
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i+1, r):
            indices[j] = indices[j-1] + 1
        yield tuple(pool[i] for i in indices)

In [ ]:
for it in myIter(iterable, r):
    print(it)

In [ ]:
class myDict(dict):
    def __getattr__(self,key):
        return self.get(key)
 
    def __setattr__(self,key,value):
        self[key] = value

In [ ]:
t = myDict()

In [ ]:
t.a0 = 4

In [ ]:
t['a0']

# Scene reference point

In [3]:
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [8]:
import sceneReferencePoints

SyntaxError: invalid syntax (<ipython-input-8-6f8112f30e86>, line 1)

In [5]:
myVars = [var for var in dir(sceneReferencePoints) if var[:2] != '__' and var[-2:] != '__']
refs = {}
for var in myVars:
    refs[var] = sceneReferencePoints.__dict__[var]

In [6]:
refs

{'corner': [48.0577, -2.00845],
 'h0': [48.05681666666667, -2.0078],
 'h1': [48.05681666666667, -2.008016666666667],
 'h2': [48.05668333333333, -2.0079833333333332],
 'h3': [48.05668333333333, -2.0077833333333333],
 'p0': [48.05755, -2.010483333333333],
 'p1': [48.05733333333333, -2.0104],
 'p2': [48.0584, -2.0059666666666667],
 'p3': [48.05818333333333, -2.0058833333333332],
 's0': [48.06081666666667, -1.9950166666666667],
 's1': [48.060833333333335, -2.0218833333333333],
 's2': [48.054566666666666, -2.0225666666666666],
 's3': [48.05458333333333, -1.9954166666666668],
 's4': [48.066716666666665, -2.0132166666666667],
 's5': [48.049, -2.0132166666666667],
 's6': [48.0486, -2.0038],
 's7': [48.066916666666664, -2.0037666666666665],
 'x0': [48.06101666666667, -1.98155],
 'x1': [48.06078333333333, -2.03535],
 'x2': [48.05455, -2.0356833333333335],
 'x3': [48.054633333333335, -1.9822166666666665],
 'x4': [48.0762, -2.0133],
 'x5': [48.040166666666664, -2.01325],
 'x6': [48.0399, -2.003766

In [7]:
t[var[0]]

NameError: name 't' is not defined

In [9]:
import importlib

In [12]:
sys.path.insert(0, "/home/pleroy/DATA/PoSAR-X/PIMA-1")

In [19]:
# import scene reference points
refsFile = "sceneReferencePoints"
sceneReferencePoints = importlib.import_module(refsFile)
refs = epsg.getReferencePoints(sceneReferencePoints)
refsEpsg = {}
for key in refs:
    refsEpsg[key + "_epsg"] = epsg.wgs84LongLatToEpsg( refs[key][::-1], epsg.epsg3857 )
print(f"{len(refsEpsg)} reference points converted from Long-Lat to EPSG")


25 reference points converted from Long-Lat to EPSG


In [21]:
refsFile.split('.py')[0]

'sceneReferencePoints'

## Look for prefixes in the reference points

In [43]:
import re

In [82]:
def getPrefixes(varDict):
    prefixList = [var.rstrip('0123456789') for var in varDict]
    prefixDict = dict.fromkeys(prefixList, 0)
    for var in varDict:
        prefix = var.rstrip('0123456789')
        p = re.compile(f"^{prefix}\d*$")
        if p.match(var):
            prefixDict[prefix] += 1
    return prefixDict

In [83]:
prefixDict = getPrefixes(refs)

In [84]:
prefixDict

{'corner': 1, 'h': 4, 'p': 4, 's': 8, 'x': 8}

In [80]:
print(f"prefix {prefix}")
for ref in refs:
    if p.match(ref):
        print(ref)
        prefixDict[ref] += 1

prefix corner
corner


In [81]:
prefixDict

{'corner': 1, 'h': 0, 'p': 0, 's': 0, 'x': 0}

In [112]:
sceneReferencePoints = importlib.import_module("sceneReferencePoints")
pts = epsg.getReferencePoints(sceneReferencePoints)
prefixes = epsg.getReferencePointPrefixes(pts)
ptsEpsg = {pt: epsg.wgs84LongLatToEpsg(pts[pt][::-1], epsg.epsg3857)
           for pt in pts}
print(f"{len(ptsEpsg)} reference points converted from Long-Lat to EPSG")
print(prefixes)

25 reference points converted from Long-Lat to EPSG
{'corner': 1, 'h': 4, 'p': 4, 's': 8, 'x': 8}


In [116]:
for prefix in prefixes:
    print(f"{prefix}")
    print( [ptsEpsg[pt] for pt in pts if pt.rstrip('0123456789') == prefix] )

corner
[(-223579.63128375093, 6116459.432696062)]
h
[(-223507.27361473528, 6116312.3141455), (-223531.39283774048, 6116312.3141455), (-223527.68218804782, 6116290.107791078), (-223505.41828988894, 6116290.107791078)]
p
[(-223805.98091502956, 6116434.450122764), (-223796.70429079788, 6116398.364312055), (-223303.18788161268, 6116576.019000518), (-223293.911257381, 6116539.932594035)]
s
[(-222084.23945742747, 6116978.531520866), (-225075.0231100744, 6116981.307534891), (-225151.0914287827, 6115937.589617146), (-222128.7672537452, 6115940.365294195), (-224110.2541898657, 6117961.29664525), (-224110.2541898657, 6115010.563740962), (-223061.99565156052, 6114943.954962043), (-223058.28500186783, 6117994.612691343)]
x
[(-220585.1369814113, 6117011.843748446), (-226574.12558609052, 6116972.979495519), (-226611.23208302012, 6115934.813940995), (-220659.3499752733, 6115948.692330736), (-224119.53081409738, 6119541.174989634), (-224113.96483955838, 6113539.740288336), (-223058.28500186783, 611349

In [115]:
[pt for pt in pts]

['corner',
 'h0',
 'h1',
 'h2',
 'h3',
 'p0',
 'p1',
 'p2',
 'p3',
 's0',
 's1',
 's2',
 's3',
 's4',
 's5',
 's6',
 's7',
 'x0',
 'x1',
 'x2',
 'x3',
 'x4',
 'x5',
 'x6',
 'x7']

# Logging

In [100]:
import logging

In [101]:
getattr(logging, loglevel)

NameError: name 'loglevel' is not defined

In [102]:
logging.lastResort

<_StderrHandler stderr (WARNING)>

In [103]:
__name__

'__main__'

# Load GPS Log

In [119]:
iniFile = "/home/pleroy/DATA/PoSAR-X/PIMA-1/flight1.ini"
refs, (dir_sbg, session, sbg_hours), (dir_posar, day, hours) = check_ini(iniFile)
prefix = f"{dir_sbg}/{session}/{day}/"
nbHours = len(sbg_hours)
logEkfEuler = [f"{prefix}{h}/sbgLogEkfEuler.dat" for h in sbg_hours ]
logEkfNav   = [f"{prefix}{h}/sbgLogEkfNav.dat"   for h in sbg_hours ]
logUtcData  = [f"{prefix}{h}/sbgLogUtcData.dat"  for h in sbg_hours ]
logGpsPos   = [f"{prefix}{h}/sbgLogGpsPos.csv"   for h in sbg_hours ]
logGpsVel   = [f"{prefix}{h}/sbgLogGpsVel.dat"   for h in sbg_hours ]
logEventB   = [f"{prefix}{h}/sbgLogEventB.dat"   for h in sbg_hours ]

In [120]:
gps = ekf.loadLogGpsPos(logGpsPos, nbHours)

In [130]:
gps[0].shape, gps[1].shape

((2240, 6), (10735, 6))

In [145]:
gps_all = np.vstack(gps)

In [146]:
gps_all.shape

(12975, 6)

In [138]:
10735*6+2240*6

77850